# 1. Install and Import Dependencies

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Gramformer.git

In [3]:
from gramformer import Gramformer

# 2. Instantiate Gramformer

In [4]:
gf = Gramformer(models=3, use_gpu=True) # 0 = detector, 1 = highlighter, 2 = corrector, 3 = all

[Gramformer] All models loaded..


# 3. Run Correction

In [8]:
gf.correct('My camera battery a dead')

['My camera battery is dead']

In [10]:
sentences = [
    'I like for walks', 
    'World is flat', 
    'Red a color', 
    'I wish my Computer was run faster.'
]

In [12]:
for sentence in sentences:
    res = gf.correct(sentence)
    print(res[0])

I like to go for walks
The world is flat
Red.
I wish my Computer would run faster.


# 4. Putting it Together with Gradio

In [ ]:
!pip install gradio

In [14]:
import gradio as gr

In [15]:
def correct(sentence):
    res = gf.correct(sentence) # Gramformer correct
    return res[0] # Return first value in res array

In [22]:
app_inputs = gr.inputs.Textbox(lines=2, placeholder="Enter sentence here...")

In [23]:
interface = gr.Interface(fn=correct, 
                        inputs=app_inputs,
                         outputs='text', 
                        title='Sup, I\'m Gramformer')

In [24]:
interface.launch()

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)

In [ ]:
gf = Gramformer(models=1, use_gpu=True) # 0 = detector, 1 = highlighter, 2 = corrector, 3 = all
### extract the  str between string symbol "{""" and ""}""
def extract_str(text):
    text=str(text)
    start = text.find("{'")
    end = text.find("'}")
    return text[start+2:end]
    
def correct(sentence):
    res = gf.correct(sentence) # Gramformer correct
    return res # Return first value in res array

def translate_transformers(model_choice, from_text):
    # initialize pipeline with the translation model we want to use
    if model_choice == "English to Chinese":
        translation_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh")
        res = translation_pipeline(from_text)[0]      
        return res['translation_text']
    elif model_choice == "Chinese to English":
        translation_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-zh-en")
        res = translation_pipeline(from_text)[0]      
        return res['translation_text']
    elif model_choice =="Gramacheck":
        translation_pipeline = correct(from_text)
        return extract_str(translation_pipeline)
    

interface = gr.Interface(
    fn=translate_transformers,
    title="Choose language translation",
    inputs=[
        gr.inputs.Dropdown(["English to Chinese", "Chinese to English", "Gramacheck"]),
        # gr.inputs.Textbox(lines=2, placeholder='Text to translate'),
        gr.inputs.Textbox(lines=4, placeholder="Enter sentence here...")
    ],
    outputs="text"
)

interface.launch()